In [1]:
import pandas as pd
import numpy as np
import random
from random import randrange
import time
import os
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
import itertools
from itertools import permutations
from itertools import combinations
from itertools import chain

import multiprocessing
from multiprocessing import Process
from multiprocessing import Pool

import cProfile
import pstats
import io
from pstats import SortKey
from functools import wraps

from copy import deepcopy

import json

import collections
import statistics

# https://www.redblobgames.com/grids/circle-drawing/

In [2]:
def timing_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

#@timing_decorator

In [3]:
def export_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    print(f"Data exported to {filename}")


def import_from_json(filename):
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
    return data



In [4]:
import DFS_Functions
import DFS_World_States

In [5]:
from DFS_Functions import adjacent_locations, chebyshev_distance, bresenham_line, calculate_full_path, check_opportunity_attacks, is_line_of_sight_clear, check_visibility
from DFS_World_States import world, world_grid_states

In [6]:
# statistics and probabilities
# chance to hit = (21 + Attack Bonus - Target AC) / 20
# chance to hit disadvantage = (21 + Attack Bonus - Target AC)^2 / 400
# chance to hit advantage = 1 - (Target AC - Attack Bonus - 1)^2 / 400


In [7]:
class entity:
    def __init__(self,world):
        self.world = world
        self.is_spawned = False

        self.caster = False
        self.concentrating = False
        self.concentration = False

        self.attack_limit = 1
        self.weapon_equipped = []
        self.equipped_armor = []
        self.inventory = []

        self.ac = 10
        self.hp = 1
        self.conditions = []

        self.circumstances = {}

        self.spells = {}

        if self.is_spawned == False:
            self.location = [0,0]
            
        elif self.is_spawned == True:
            x_loc = world.grid[np.where(world.grid[:,0] == max(world.grid))]
            print('x: ',x_loc)
            y_loc = world.grid[np.where(world.grid[0,:] == max(world.grid))]
            print('y: ',y_loc)
            self.location = [x_loc,y_loc]

        self.speed = 6
        self.coins = 0

        # will need to have a add_entity() in order to add a key: value pair to circumstances per enemy in world


In [8]:
start_time = time.time()

In [9]:
stage = world(11)
stage.generate_map()
actor = entity(stage)
stage.add_enemy((1,1))
stage.add_enemy((-2,0))
stage.add_enemy((0,3))
stage.add_enemy((2,2))
stage.add_enemy((-3,-3))
stage.add_enemy((1,-1))



stage.add_coin((3,0))
stage.add_coin((-1,-1))
stage.add_coin((1,1))

#stage.visualize()

In [10]:
# This generates the action series

class RuleBasedSequenceDFS:
    def __init__(self, min_length, max_length, start, end, rules, acting_entity):
        self.min_length = min_length
        self.max_length = max_length
        self.start = start
        self.end = end
        self.rules = rules
        self.sequences = []
        self.acting_entity = acting_entity

    def check_rules(self, sequence, next_num, acting_entity):
        # Apply all rules to the current sequence and next number
        return all(rule(sequence, next_num, acting_entity) for rule in self.rules)

    def dfs(self, current_sequence):
        # Check if the current sequence is within the desired length range
        if self.min_length <= len(current_sequence) <= self.max_length:
            self.sequences.append(current_sequence.copy())
        
        # Continue exploring if we haven't reached the maximum length
        if len(current_sequence) < self.max_length:
            for next_num in range(self.start, self.end + 1):
                if self.check_rules(current_sequence, next_num, self.acting_entity):
                    current_sequence.append(next_num)
                    self.dfs(current_sequence)
                    current_sequence.pop()  # Backtrack


    def generate_sequences(self):
        self.dfs([])
        return self.sequences





In [94]:

subaction_dict = {
    '0': 'Move_One',                # needs a target
    '1': 'Move_Two',
    '2': 'Move_Three',
    '3': 'Move_Four',
    '4': 'Object (Free) Ground',      # needs a target
    '5': 'Attack',              # needs a target
    '6': 'Dodge',
    '7': 'Object (Action) Ground',              # needs a target
    '8': 'Disengage',
    '9': 'Hide',
    '10': 'Help',
    '11': 'Cast',
    '12': 'Dash',
    '13': 'Don or Doff Shield',
    '14': 'End Concentration',
    '15': 'Off Hand Weapon Attack',
    '16': 'Grapple',
    '17': 'Escape Grapple',
    '18': 'Shove',
    '19': 'Go Prone',
    '20': 'Stand Up',
    '21': 'Stabilize Creature',
    '22': 'Wake Creature',
    '23': 'Move_Five',
    #'24': 'Move_Six',
    #'25': 'Object (Free) Self',
    #'26': 'Object (Action) Self',
    #'27': 'Object (Free) Environment',
    #'28': 'Object (Action) Environment',

}

# object interaction may need to be fleshed out
# - object in inventory
# - object in environment
# - environment interaction



move_subactions = [0,1,2,3,23,24]
    # jump
    # crawl
    # squeeze

action_subactions = [5,6,7,8,9,10,11,12,13,16,17,18,21,22,26,28]
attack_subactions = [5,16,18]
free_subactions = [4,14]               # End Concentration will go here as well
bonus_subactions = [15]
object_subactions = [4,7,25,26,27,28]
object_action_subactions = [7,26,28]
object_free_subactions = [4,25,27]

subactions_req_targets = [0,1,2,3,4,5,7,10,15,23,24,27,28]
subactions_req_allies = [10,21,22]

theoretical_turn_length = len(free_subactions) + 6 + 1 + 1

# rules
# - only one action
def rule_only_one_action(sequence, next_num, acting_entity):
    if next_num in action_subactions and sum(1 for seq in sequence if seq in action_subactions) == 1:
        return False
    return True

def rule_only_one_bonus_action(sequence, next_num, acting_entity):
    if next_num in bonus_subactions and sum(1 for seq in sequence if seq in bonus_subactions) == 1:
        return False
    return True

def rule_no_redundant_moves(sequence, next_num, acting_entity):
    if len(sequence) > 0:
        if next_num == 0 and sequence[-1] == 0:
            return False
        if next_num == 1 and sequence[-1] == 1:
            return False
        if next_num == 2 and sequence[-1] == 2:
            return False
        if next_num == 3 and sequence[-1] == 3:
            return False
        
        if next_num == 0 and sequence[-1] == 1:
            return False
        if next_num == 1 and sequence[-1] == 0:
            return False
        
        if next_num == 1 and sequence[-1] == 2:
            return False
        if next_num == 2 and sequence[-1] == 1:
            return False
        
        if next_num == 0 and sequence[-1] == 2:
            return False
        if next_num == 2 and sequence[-1] == 0:
            return False
        
        if next_num == 0 and sequence[-1] == 3:
            return False
        if next_num == 3 and sequence[-1] == 0:
            return False
        
        if next_num == 0 and sequence[-1] == 23:
            return False
        if next_num == 23 and sequence[-1] == 0:
            return False
        
        if next_num == 2 and sequence[-1] == 3:
            return False
        if next_num == 3 and sequence[-1] == 2:
            return False

    return True

def rule_limited_move_speed(sequence, next_num, acting_entity):
    move_speed = acting_entity.speed
    move_points = {
                0: 1, 
                1: 2, 
                2: 3, 
                3: 4, 
                20: move_speed/2,
                23: 5,
                24: 6,
                }

    speed_spent = sum(move_points[move_type] for move_type in sequence if move_type in move_subactions)
    
    # if prone is taken, stand up hasn't been, multiply the costs of move_points by 2
    if 19 in sequence:
        # if between the last time 19 was called, there isn't a 20, false
        last_prone_index = len(sequence) - 1 - sequence[::-1].index(19)

        if 20 not in sequence[last_prone_index:]:
            if next_num in move_subactions:
                if speed_spent + (move_points[next_num]*2) > move_speed:
                    return False

    if next_num in move_subactions:
        if speed_spent + move_points[next_num] > move_speed:
            return False

    return True

def rule_one_of_each_free_action(sequence, next_num, acting_entity):
    if next_num in free_subactions and sum(1 for seq in sequence if seq == next_num) == 1:
        return False
    return True

def rule_shield(sequence, next_num, acting_entity):
    if next_num == 13 and 'shield' not in acting_entity.inventory or next_num == 13 and 'shield' not in acting_entity.equipped_armor:
        return False
    return True

def rule_concentration(sequence, next_num, acting_entity):
    if next_num == 14 and acting_entity.concentration == False:
        return False
    return True

def rule_actions_requiring_allies(sequence, next_num, acting_entity):
    if acting_entity.world.non_enemies == []:
        if next_num in subactions_req_allies:
            return False

    return True

def rule_off_hand_two_weapons_requirement(sequence, next_num, acting_entity):
    if next_num == 15 and len(acting_entity.weapon_equipped) < 2:
        return False
    return True

def rule_condition_rules(sequence, next_num, acting_entity):
    # escape grappled
    if next_num == 17 and 'grappled' not in acting_entity.conditions:
        return False
    
    # go prone
    if next_num == 19 and 'prone' in acting_entity.conditions:
        return False
    
    # stand up
    if next_num == 20 and 'grappled' in acting_entity.conditions:
        return False

    return True

def rule_standing_up_rules(sequence, next_num, acting_entity):
    # can't stand up twice
    if 20 in sequence and next_num == 20:
        return False
    
    if next_num == 20 and 19 not in sequence:
        return False

    return True

def rule_remove_redundant_prones(sequence, next_num, acting_entity):
    if len(sequence)>0:
        if next_num == 19 and sequence[-1] == 19:
            return False
        
        if 19 in sequence:
            # if between the last time 19 was called, there isn't a 20, false
            last_prone_index = len(sequence) - 1 - sequence[::-1].index(19)

            if 20 not in sequence[last_prone_index:] and next_num == 19:
                return False

    return True

def rule_remove_redundant_objects(sequence, next_num, acting_entity):
    if next_num in object_action_subactions:
        if any(item in object_action_subactions for item in sequence):
            return False

    if next_num in object_free_subactions:
        if any(item in object_free_subactions for item in sequence):
            return False

    return True

rules = [rule_only_one_action, 
         rule_only_one_bonus_action, 
         rule_no_redundant_moves,
         rule_limited_move_speed,
         rule_one_of_each_free_action,
         rule_shield,
         rule_concentration,
         rule_actions_requiring_allies,
         rule_off_hand_two_weapons_requirement,
         rule_condition_rules,
         rule_standing_up_rules,
         rule_remove_redundant_prones,
         ]



# Generate sequences of length 3, with numbers from 1 to 15, with no consecutive numbers and prime sum
action_sequence_generator = RuleBasedSequenceDFS(min_length=0,max_length=theoretical_turn_length, start=0, end=len(subaction_dict), rules=rules, acting_entity=actor)
action_series_full_list = action_sequence_generator.generate_sequences()


In [ ]:
sum(1 for s in action_series_full_list if isinstance(s,list) and any(isinstance(item, list) and 5 in item for item in s))

In [44]:
'''
subaction_dict = {
    '0': 'Move_One',                # needs a target
    '1': 'Move_Two',
    '2': 'Move_Three',
    '3': 'Move_Four',
    '4': 'Object (Free)',      # needs a target
    '5': 'Attack',              # needs a target
    '6': 'Dodge',
    '7': 'Object',              # needs a target
    '8': 'Disengage',
    '9': 'Hide',
    '10': 'Help',
    '11': 'Cast',
    '12': 'Dash',
    '13': 'Don or Doff Shield',
    '14': 'End Concentration',
    '15': 'Off Hand Weapon Attack',
    '16': 'Grapple',
    '17': 'Escape Grapple',
    '18': 'Shove',
    '19': 'Go Prone',
    '20': 'Stand Up',
    '21': 'Stabilize Creature',
    '22': 'Wake Creature',
    '23': 'Move_Five',
    #'24': 'Move_Six',

}

move_subactions = [0,1,2,3,23,24]
    # go prone
    # stand up
    # jump
    # crawl
    # squeeze

action_subactions = [5,6,7,8,9,10,11,12,13,16,17,18,21,22]
attack_subactions = [5,16,18]
free_subactions = [4,14]               # End Concentration will go here as well
bonus_subactions = [15]

subactions_req_targets = [0,1,2,3,4,5,7,10,15,23,24]
subactions_req_allies = [10,21,22]

theoretical_turn_length = len(free_subactions) + 6 + 1 + 1

# rules
# - only one action
def rule_only_one_action(sequence, next_num, acting_entity):
    if next_num in action_subactions and sum(1 for seq in sequence if seq in action_subactions) == 1:
        return False
    return True

def rule_only_one_bonus_action(sequence, next_num, acting_entity):
    if next_num in bonus_subactions and sum(1 for seq in sequence if seq in bonus_subactions) == 1:
        return False
    return True

def rule_no_redundant_moves(sequence, next_num, acting_entity):
    if len(sequence) > 0:
        if next_num == 0 and sequence[-1] == 0:
            return False
        if next_num == 1 and sequence[-1] == 1:
            return False
        if next_num == 2 and sequence[-1] == 2:
            return False
        if next_num == 3 and sequence[-1] == 3:
            return False
        
        if next_num == 0 and sequence[-1] == 1:
            return False
        if next_num == 1 and sequence[-1] == 0:
            return False
        
        if next_num == 1 and sequence[-1] == 2:
            return False
        if next_num == 2 and sequence[-1] == 1:
            return False
        
        if next_num == 0 and sequence[-1] == 2:
            return False
        if next_num == 2 and sequence[-1] == 0:
            return False
        
        if next_num == 0 and sequence[-1] == 3:
            return False
        if next_num == 3 and sequence[-1] == 0:
            return False
        
        if next_num == 0 and sequence[-1] == 23:
            return False
        if next_num == 23 and sequence[-1] == 0:
            return False
        
        if next_num == 2 and sequence[-1] == 3:
            return False
        if next_num == 3 and sequence[-1] == 2:
            return False

    return True

def rule_limited_move_speed(sequence, next_num, acting_entity):
    move_speed = acting_entity.speed
    move_points = {
                0: 1, 
                1: 2, 
                2: 3, 
                3: 4, 
                20: move_speed/2,
                23: 5,
                24: 6,
                }

    speed_spent = sum(move_points[move_type] for move_type in sequence if move_type in move_subactions)
    
    # if prone is taken, stand up hasn't been, multiply the costs of move_points by 2
    if 19 in sequence:
        # if between the last time 19 was called, there isn't a 20, false
        last_prone_index = len(sequence) - 1 - sequence[::-1].index(19)

        if 20 not in sequence[last_prone_index:]:
            if next_num in move_subactions:
                if speed_spent + (move_points[next_num]*2) > move_speed:
                    return False

    if next_num in move_subactions:
        if speed_spent + move_points[next_num] > move_speed:
            return False

    return True

def rule_one_of_each_free_action(sequence, next_num, acting_entity):
    if next_num in free_subactions and sum(1 for seq in sequence if seq == next_num) == 1:
        return False
    return True

def rule_shield(sequence, next_num, acting_entity):
    if next_num == 13 and 'shield' not in acting_entity.inventory or next_num == 13 and 'shield' not in acting_entity.equipped_armor:
        return False
    return True

def rule_concentration(sequence, next_num, acting_entity):
    if next_num == 14 and acting_entity.concentration == False:
        return False
    return True

def rule_actions_requiring_allies(sequence, next_num, acting_entity):
    if acting_entity.world.non_enemies == []:
        if next_num in subactions_req_allies:
            return False

    return True

def rule_off_hand_two_weapons_requirement(sequence, next_num, acting_entity):
    if next_num == 15 and len(acting_entity.weapon_equipped) < 2:
        return False
    return True

def rule_condition_rules(sequence, next_num, acting_entity):
    # escape grappled
    if next_num == 17 and 'grappled' not in acting_entity.conditions:
        return False
    
    # go prone
    if next_num == 19 and 'prone' in acting_entity.conditions:
        return False
    
    # stand up
    if next_num == 20 and 'grappled' in acting_entity.conditions:
        return False

    return True

def rule_standing_up_rules(sequence, next_num, acting_entity):
    # can't stand up twice
    if 20 in sequence and next_num == 20:
        return False
    
    if next_num == 20 and 19 not in sequence:
        return False

    return True

def rule_remove_redundant_prones(sequence, next_num, acting_entity):
    if len(sequence)>0:
        if next_num == 19 and sequence[-1] == 19:
            return False
        
        if 19 in sequence:
            # if between the last time 19 was called, there isn't a 20, false
            last_prone_index = len(sequence) - 1 - sequence[::-1].index(19)

            if 20 not in sequence[last_prone_index:] and next_num == 19:
                return False

    return True


rules_zero = [rule_only_one_action, 
         rule_only_one_bonus_action, 
         rule_no_redundant_moves,
         rule_limited_move_speed,
         rule_one_of_each_free_action,
         rule_shield,
         rule_concentration,
         rule_actions_requiring_allies,
         rule_off_hand_two_weapons_requirement,
         rule_condition_rules,
         rule_standing_up_rules,
         rule_remove_redundant_prones,
         ]


'''



"\nsubaction_dict = {\n    '0': 'Move_One',                # needs a target\n    '1': 'Move_Two',\n    '2': 'Move_Three',\n    '3': 'Move_Four',\n    '4': 'Object (Free)',      # needs a target\n    '5': 'Attack',              # needs a target\n    '6': 'Dodge',\n    '7': 'Object',              # needs a target\n    '8': 'Disengage',\n    '9': 'Hide',\n    '10': 'Help',\n    '11': 'Cast',\n    '12': 'Dash',\n    '13': 'Don or Doff Shield',\n    '14': 'End Concentration',\n    '15': 'Off Hand Weapon Attack',\n    '16': 'Grapple',\n    '17': 'Escape Grapple',\n    '18': 'Shove',\n    '19': 'Go Prone',\n    '20': 'Stand Up',\n    '21': 'Stabilize Creature',\n    '22': 'Wake Creature',\n    '23': 'Move_Five',\n    #'24': 'Move_Six',\n\n}\n\nmove_subactions = [0,1,2,3,23,24]\n    # go prone\n    # stand up\n    # jump\n    # crawl\n    # squeeze\n\naction_subactions = [5,6,7,8,9,10,11,12,13,16,17,18,21,22]\nattack_subactions = [5,16,18]\nfree_subactions = [4,14]               # End Concentr

In [45]:
def precalc_reward(action_series, entity):
    reward_value = 0
    act_series_len = len(action_series)


    # if an item from action_subactions is in action_series:
    for i in [True for item in action_series if item in action_subactions]:
        reward_value += 1
    
    for i in [True for item in action_series if item in free_subactions]:
        reward_value += 1

    for i in [True for item in action_series if item in attack_subactions]:
        reward_value += 1


    #if entity.location in entity.world.enemy_adjacent_locations and 8 not in action_series and any(0 in action_series or 1 in action_series or 2 in action_series or 3 in action_series):
    #    reward_value -= 1
    
    if 19 in action_series:
        reward_value -= 1

    # if the entity ends the turn while prone
    if 19 in action_series:
        last_prone_index = len(action_series) - 1 - action_series[::-1].index(19)
        if 20 not in action_series[last_prone_index:]:
            reward_value -= 1

    # if the action series is between 3 and 6, reward else punish
    if act_series_len >= 3:
        reward_value += 1
    
    if act_series_len <= 7:
        reward_value += 1

    if act_series_len > 8:
        reward_value -= 1

    if 5 not in action_series:
        reward_value -= 1
    elif 6 not in action_series:
        reward_value -= 1
    elif 8 not in action_series:
        reward_value -= 1
    elif 9 not in action_series:
        reward_value -= 1
    else:
        pass

    
    



    return reward_value

reward_series_full_list = [precalc_reward(i, actor) for i in action_series_full_list]




In [46]:
#def function_gen_dfs_loc(action_series,entity):
#    location_sequence_generator = RuleBasedLocationSequenceDFS(length=sum(x for x in action_series if x in subactions_req_targets),
#                                                               start=0,
#                                                               end=
#                                                               )

#    return sequence

#location_series_dict = {function_gen_dfs_loc(action_series) for action_series in action_series_full_list if reward_series_full_list[action_series_full_list.index(action_series)] > 0}


In [47]:
target_distance_scores = { # the distance from which a location can be per subaction
            0: 1,
            1: 2, 
            2: 3,
            3: 4,
            5: 1,
            4: 1,
            7: 1,
            10: 1,
            15: 1,
            23: 5,
            24: 6
        }

In [48]:
# this one handles location series
class RuleBasedLocationSequenceDFS2:
    def __init__(self, action_series_list, reward_series_list, target_distance_scores, location_rules, acting_entity):
        self.action_series_list = action_series_list
        self.reward_series_list = reward_series_list
        self.target_distance_scores = target_distance_scores
        self.location_rules = location_rules
        self.acting_entity = acting_entity

        self.location_series_list = []
        
        self.sequences = []
        


    # I think what I want to have happen is that self.sequences will be filled per action_series 
    # then appended to location_series_list 
    # then reset and used for the next action_series

    def get_potential_locations(self, sub_index, tar_act_series, current_sequence, acting_entity, grid_locations):
        # identify the distance the subaction allows
        #print('')
        #print('--entered potential locations--')
        subaction = tar_act_series[sub_index]
        distance_allowable = target_distance_scores[subaction]

        # identify the entity's location at that given subaction
        entity_loc_series = [current_sequence[loc_index] for loc_index in range(len(current_sequence)) if tar_act_series[loc_index] in move_subactions]
        
        #print(f'Entity_loc_series: {entity_loc_series}')
        if len(entity_loc_series) < 1:
            entity_location = acting_entity.location
        else:
            entity_location = entity_loc_series[-1]


        # identify which locations/spaces can be accessed within that distance
        locations_accessible = []
        for loc in grid_locations:
            if chebyshev_distance(entity_location, loc) <= distance_allowable:
                locations_accessible.append(loc)

        #print(f'locations accessible {locations_accessible}')
        #print(f'entity is at {entity_location}; allowable distance for action {subaction} is {distance_allowable}; locations accessible are {locations_accessible}')

        #print(f'Locations accessible: {locations_accessible}')
        #print(' ')
        return locations_accessible


    def check_rules(self, sequence, next_loc, acting_entity, action_series):
        return all(location_rule(sequence, next_loc, acting_entity, action_series) for location_rule in self.location_rules)


    def dfs(self, action_series, current_sequence, grid_locations):
        tar_act_series = [i for i in action_series if i in subactions_req_targets]
        required_sequence_length = len(tar_act_series)
        
        if len(current_sequence) == required_sequence_length:
            #print(f'finishing if current sequence: {current_sequence}')
            return [current_sequence.copy()]  # Return a list containing the completed sequence
        
        all_sequences = []
        sub_index = len(current_sequence)  # This ensures we're looking at the correct subaction
        
        locations_accessible = self.get_potential_locations(
            sub_index=sub_index,
            tar_act_series=tar_act_series,
            current_sequence=current_sequence,
            acting_entity=self.acting_entity,
            grid_locations=grid_locations
        )

        for next_loc in locations_accessible:
            if self.check_rules(current_sequence, next_loc, self.acting_entity, tar_act_series):
                current_sequence.append(next_loc)
                all_sequences.extend(self.dfs(action_series, current_sequence, grid_locations))
                current_sequence.pop()
        
        #print(f'sequences being returned from dfs to results 1 {all_sequences[:20]}')
        return all_sequences

    def generate_sequences(self):
        grid_locations = [
            [x - 5, y - 5]
            for x in range(len(self.acting_entity.world.grid))
            for y in range(len(self.acting_entity.world.grid[x]))
        ]

        for action_series_index, action_series in enumerate(self.action_series_list):
            if self.reward_series_list[action_series_index] >= 4:
                #print('')
                #print('---------------------')
                #print(f'{action_series}: {self.reward_series_list[action_series_index]}')
                
                sequences = self.dfs(
                    action_series=action_series,
                    current_sequence=[],
                    grid_locations=grid_locations
                )
                
                self.location_series_list.append(sequences)
            
            else: 
                self.location_series_list.append([])
        
        #print(f'Finished generate sequences: {self.location_series_list}')
        return self.location_series_list





In [49]:
def rule_location_spacing_rule(sequence, next_loc, acting_entity, action_series):
    tar_act_req = [x for x in action_series if x in subactions_req_targets]
    move_points = {
            0: 1, 
            1: 2, 
            2: 3, 
            3: 4, 
            20: acting_entity.speed/2,
            23: 5,
            24: 6,
            }


    #act_loc_zip = zip(tar_act_req,sequence)
    #for i in range(len(tar_act_req)):
    # if next_loc is further away from the previous location than the action allows, false
    #if chebyshev_distancenext_loc...move_points[tar_act_req[len(sequence)+1]]
    # is this one even needed because of the locations_includable???
    return True

def rule_no_unaddressed_location_series(sequence, next_loc, acting_entity, action_series):
    tar_act_req = [x for x in action_series if x in subactions_req_targets]
    if len(sequence) + 1 < len(tar_act_req):
        print('no unaddressed locations')
        return False
    return True

def rule_no_excess_locations(sequence, next_loc, acting_entity, action_series):
    tar_act_req = [x for x in action_series if x in subactions_req_targets]
    if len(sequence) + 1 > len(tar_act_req):
        print('no excess locations')
        return False
    return True

def rule_no_repeating_locations(sequence, next_loc, acting_entity, action_series):
    tar_act_req = [x for x in action_series if x in subactions_req_targets]
    for act_index in range(len(sequence)):
        loc = sequence[act_index]
        act = sequence[act_index]

        if act in move_subactions and next_loc == loc:
            #print('no repeating move locations')
            return False

    return True


def rule_attacks_target_enemies(sequence, next_loc, acting_entity, action_series):

    # if tar_act_req[action_index] == 5:
    #    print(f'sequence {sequence}')
    #    print(f'action_index for next_loc ({next_loc}): {action_index}; action: {tar_act_req[action_index]}')
    #    print(f'tar_act_req: {tar_act_req}')

    tar_act_req = [x for x in action_series if x in subactions_req_targets]
    entity_loc_series = [sequence[loc_index] for loc_index in range(len(sequence)) if tar_act_req[loc_index] in move_subactions]
        #print(f'Entity_loc_series: {entity_loc_series}')
    if len(entity_loc_series) < 1:
        entity_location = acting_entity.location
    else:
        entity_location = entity_loc_series[-1]
    action_index = len(sequence)

    print(acting_entity.world.enemy_locations)
    print(next_loc)
    if tar_act_req[action_index] in attack_subactions:
        if next_loc not in acting_entity.world.enemy_locations:
            #print('attack does not target enemies')
            print('false')
            return False
    
    #print('passed the enemy test')
    return True


def rule_objects_target_objects(sequence, next_loc, acting_entity, action_series):
    tar_act_req = [x for x in action_series if x in subactions_req_targets]
    entity_loc_series = [sequence[loc_index] for loc_index in range(len(sequence)) if tar_act_req[loc_index] in move_subactions]
        #print(f'Entity_loc_series: {entity_loc_series}')
    if len(entity_loc_series) < 1:
        entity_location = acting_entity.location
    else:
        entity_location = entity_loc_series[-1]
    action_index = len(sequence)

    #print(f'item locations: {acting_entity.world.coin_locations}')

    if tar_act_req[action_index] in attack_subactions:
        if next_loc not in acting_entity.world.coin_locations or next_loc == entity_location:
            #print('object does not target object')
            return False
    return True

def rule_hide_in_low_light(sequence, next_loc, acting_entity, action_series):
    tar_act_req = [x for x in action_series if x in subactions_req_targets]
    action_index = len(sequence)
    if tar_act_req[action_index] == 9:
        x, y = next_loc
        if acting_entity.world.grid[x][y][3] not in [2,3]:
            #print('cannot hide in bright light')
            return False
    return True


def rule_move_space_efficiency(sequence, next_loc, acting_entity, action_series):
    pass
    # essentially, the move should be efficient and not going in a circle
    # efficient movement should be calculated via vectors??? or geometry???

def rule_move_speed_efficiency(sequence, next_loc, acting_entity, action_series):
    move_points = {
        0: 1, 
        1: 2, 
        2: 3, 
        3: 4, 
        20: acting_entity.speed/2,
        23: 5,
        24: 6,
        }
    # essentially, the move shouldn't use more speed than available via difficult terrain
    # will need to calculate the sum of the move speed spent according to each loc used on a move_subaction
    tar_act_req = [x for x in action_series if x in subactions_req_targets]
    total_move_speed_spent = 0
    
    difficult_terrain = False
    for loc_index in range(len(sequence)):

        if tar_act_req[loc_index] in move_subactions:
            # causes of difficult terrain
            # - enemy creature

            if acting_entity.world.grid[sequence[loc_index]][0] == 1:
                difficult_terrain = True
            
            if acting_entity.world.grid[sequence[loc_index]][2] == 1:
                difficult_terrain = True
            
            if difficult_terrain == True:
                total_move_speed_spent += move_points[tar_act_req[loc_index]] * 2

            elif difficult_terrain == False:
                total_move_speed_spent += move_points[tar_act_req[loc_index]]
                
    if total_move_speed_spent >= acting_entity.speed:
        return False
    
    if acting_entity.world.grid[next_loc][0] == 1 or acting_entity.world.grid[sequence[loc_index]][2] == 1:
        difficult_terrain = True

    move_speed_required_for_next_action = move_points[tar_act_req[len(sequence)]]

    if difficult_terrain == True:
       move_speed_required_for_next_action = move_points[tar_act_req[len(sequence)]] * 2


    if tar_act_req[len(sequence)] in move_subactions and total_move_speed_spent + move_speed_required_for_next_action > acting_entity.speed:
        return False
    
    return True



location_rules = [
    #rule_location_spacing_rule,
    #rule_no_excess_locations,
    rule_no_repeating_locations,
    #rule_no_unaddressed_location_series,
    rule_attacks_target_enemies,
    rule_objects_target_objects,
    rule_hide_in_low_light,
    #rule_move_space_efficiency,
    #rule_move_speed_efficiency,
]

In [50]:
def analyze_reward_distribution(reward_series_full_list, action_series_full_list):
    total_count = len(reward_series_full_list)
    
    print(f'Total count: {total_count}')
    print(f'High: {max(reward_series_full_list)}')
    print(f'Low: {min(reward_series_full_list)}')
    print()

    # Count occurrences of each reward value
    reward_counts = collections.Counter(reward_series_full_list)
    
    # Calculate and print percentages for all reward values
    print("Reward Distribution:")
    for reward in sorted(reward_counts.keys()):
        count = reward_counts[reward]
        percentage = (count / total_count) * 100
        print(f'% of {reward}s: {percentage:.2f}%')
    print()

    print(f"Total Qualifiers: {len([x for x in reward_series_full_list if x >= 4])}")
    print()

    # Print individual rewards and actions
    #print("Individual Rewards and Actions:")
    #for i, (reward, action) in enumerate(zip(reward_series_full_list, action_series_full_list)):
    #    print(f'{i}: {reward} - {action}')

In [51]:
analyze_reward_distribution(reward_series_full_list, action_series_full_list)

Total count: 141903
High: 4
Low: -2

Reward Distribution:
% of -2s: 0.03%
% of -1s: 8.64%
% of 0s: 25.14%
% of 1s: 32.03%
% of 2s: 25.63%
% of 3s: 8.17%
% of 4s: 0.38%

Total Qualifiers: 534



In [52]:
location_series_generator = RuleBasedLocationSequenceDFS2(action_series_list=action_series_full_list,
                                                             reward_series_list=reward_series_full_list,
                                                             location_rules=location_rules,
                                                             acting_entity=actor,
                                                             target_distance_scores=target_distance_scores)
location_series_full_list = location_series_generator.generate_sequences()

In [53]:
print(f'total: {len(location_series_full_list)}')
print(f'good: {len([x for x in reward_series_full_list if x >= 3])}')
print()

#for location_series_index in range(len(location_series_full_list)):
#    if reward_series_full_list[location_series_index] > 0:
#        print(f'{location_series_index} action series {action_series_full_list[location_series_index]}: \n reward: {reward_series_full_list[location_series_index]} \n total location_series: {len(location_series_full_list[location_series_index])} \n location_series: {location_series_full_list[location_series_index][:10]} \n')

total: 141903
good: 93935



In [54]:
def post_loc_series_reward_calc(all_action_series, all_location_series, all_reward_series, acting_entity):
    act_loc_rew_zip = list(zip(all_action_series, all_location_series, all_reward_series))
    
    #print(act_loc_rew_zip)
    post_reward_list = []
    
    for i in range(len(act_loc_rew_zip)):

        action_series = act_loc_rew_zip[i][0]
        locs_for_act = act_loc_rew_zip[i][1]
        pre_reward = act_loc_rew_zip[i][2]

        if locs_for_act != []:
            #print()
            

            #new_reward_list = [pre_reward for x in locs_for_act]

            #print(act_loc_rew_zip[i][0])
            
            for loc_series in locs_for_act:
                post_reward = pre_reward

                act_loc_rew_series = (action_series, loc_series, pre_reward)
                #print(f'act loc rew series: {act_loc_rew_series}')

                # act loc rew series: ([0, 4, 6], [[-1, -1], [-2, -2]], 0.25)
                move_act_loc = ([x for x in action_series if x in move_subactions],[loc_series[y] for y in range(len(loc_series)) if action_series[y] in move_subactions])
                #print(f'move act loc: {move_act_loc}')

                move_path = calculate_full_path(acting_entity.location, move_act_loc)
                #print(f'move path: {move_path}')

                if check_opportunity_attacks(move_path, acting_entity.world.enemy_locations):
                    post_reward -= 1
                
                #print(move_path)
                if move_path != []:   
                    vis_count = len(check_visibility(move_path[-1], acting_entity.world.enemy_locations, acting_entity.world))
                    post_reward -= vis_count


                if move_path == []:
                    post_reward -= 0.5


                # if the object action targets entity's location, but inventory is empty...
                
                # if they end their turn prone while adjacent to an enemy
                if 19 in action_series and 20 not in action_series[:-action_series.index(19)] and move_path[-1] in acting_entity.world.enemy_adjacent_locations:
                    post_reward -= 1
                
                # if the path is in a circle...
                # if the 
                

                new_act_loc_rew_series = (action_series, loc_series, post_reward)
                post_reward_list.append(new_act_loc_rew_series)            

    post_reward_list = sorted(post_reward_list, key=lambda x: x[2],reverse=True)
    return post_reward_list

            #only_move_act = [x for x in all_action_series if x in move_subactions]
            #only_move_loc = [x for x in all_location_series if all_action_series]

            #for loc_series_index in range(len(locs_for_act)):
            #    post_calc_reward = 0

                #entity_path_traveled = []
            #    move_series_zip = [(act,loc) for act, loc, rew in act_loc_rew_zip if act in move_subactions]
            #    print(move_series_zip)
            
                #move_path = calculate_full_path(acting_entity.location, list(zip()))


            #    new_reward_list[loc_series_index] += post_calc_reward

            #reward_series_full_list[i] = new_reward_list




    # if they end their turn prone adjacent to an enemy creature, punish
    # if their movement leaves a space adjacent to an enemy creature and the disengage action isn't taken, punish
    # 

In [55]:
post_reward_list = post_loc_series_reward_calc(action_series_full_list, location_series_full_list, reward_series_full_list, actor)

#for i in post_reward_list:
#    print(i)

In [56]:
post_reward_list[:20]

[([4, 16, 0], [[-1, -1], [-1, -1]], 3.5),
 ([4, 16, 0], [[-1, -1], [-1, 0]], 3.5),
 ([4, 16, 0], [[-1, -1], [-1, 1]], 3.5),
 ([4, 16, 0], [[-1, -1], [0, -1]], 3.5),
 ([4, 16, 0], [[-1, -1], [0, 0]], 3.5),
 ([4, 16, 0], [[-1, -1], [0, 1]], 3.5),
 ([4, 16, 0], [[-1, -1], [1, -1]], 3.5),
 ([4, 16, 0], [[-1, -1], [1, 0]], 3.5),
 ([4, 16, 0], [[-1, -1], [1, 1]], 3.5),
 ([4, 16, 0], [[-1, 0], [-1, -1]], 3.5),
 ([4, 16, 0], [[-1, 0], [-1, 0]], 3.5),
 ([4, 16, 0], [[-1, 0], [-1, 1]], 3.5),
 ([4, 16, 0], [[-1, 0], [0, -1]], 3.5),
 ([4, 16, 0], [[-1, 0], [0, 0]], 3.5),
 ([4, 16, 0], [[-1, 0], [0, 1]], 3.5),
 ([4, 16, 0], [[-1, 0], [1, -1]], 3.5),
 ([4, 16, 0], [[-1, 0], [1, 0]], 3.5),
 ([4, 16, 0], [[-1, 0], [1, 1]], 3.5),
 ([4, 16, 0], [[-1, 1], [-1, -1]], 3.5),
 ([4, 16, 0], [[-1, 1], [-1, 0]], 3.5),
 ([4, 16, 0], [[-1, 1], [-1, 1]], 3.5),
 ([4, 16, 0], [[-1, 1], [0, -1]], 3.5),
 ([4, 16, 0], [[-1, 1], [0, 0]], 3.5),
 ([4, 16, 0], [[-1, 1], [0, 1]], 3.5),
 ([4, 16, 0], [[-1, 1], [1, -1]], 3.5)

In [57]:
end_time = time.time()
print((end_time-start_time)/60)

1697691169.5337393


In [58]:
# how many action series have 5 in them?
sum(1 for s in post_reward_list if isinstance(s, set) and any(isinstance(item, list) and 5 in item for item in s))


0

In [59]:
print(actor.world.enemy_locations)

[(1, 1), (-2, 0), (0, 3), (2, 2), (-3, -3), (1, -1)]


In [ ]:
# ways to make things easier
# - filter by imporance
# - filter by similarity
# - filter by strategy

In [ ]:
def process_combat_zip(combat_zip):
    action_series = combat_zip[0]
    location_series = combat_zip[1]

    tar_act_series = [x for x in action_series if x in ]

In [ ]:
stage = world(10)
stage.generate_map()
actor = entity(stage)
stage.add_enemy((1,1))
stage.add_enemy((-2,0))
stage.add_enemy((0,3))

stage.add_coin((3,0))
stage.add_coin((-1,-1))

In [ ]:
stage.enemies[0]

In [ ]:
# lets create the power set for action_series

subaction_dict = {
    '0': 'Move_One',                # needs a target
    '1': 'Move_Two',
    '2': 'Move_Three',
    '3': 'Move_Four',
    '4': 'Object (Free)',      # needs a target
    '5': 'Attack',              # needs a target
    '6': 'Dodge',
    '7': 'Object',              # needs a target
    '8': 'Disengage',
    #'9': 'Hide',
    #'10': 'Help',
    #'11': 'Cast',
    #'12': 'Dash',
    #'13': 'Shield',
    #'14': 'End Concentration',
    #'15': 'Off Hand Weapon Attack',

}

entity.subaction_options_dict = subaction_dict

    # help - need to choose 1) what type of help 2) target
    # cast - need to choose 1) which spell  (and others)     2) target
    # dash - need to be able to add new spaces to the location_series...
    # don or doff shield... 
    # escape grapple
    # grapple - need to choose target, attack_subaction
    # search
    # shove - need to choose 1) target 2) prone OR knockback, attack_subaction
    # stabilize creature
    # waking someone
    
    # optional rule: Action Options 
        # Disarm
        # Overrun
        # Shove Aside
        # Tumble
        # 

move_subactions = [0,1,2,3]
    # go prone
    # stand up

action_subactions = [5,6,7,8,9,10,11,12,13]
attack_subactions = [5]
free_subactions = [4,14]               # End Concentration will go here as well
bonus_subactions = [15]

subactions_req_targets = [0,1,2,3,4,5,7,15]


# how I'll have to do Cast action, 
# it'll probably require another dictionary...
# 
# I may need another dictionary for Cast, Dash, Object, 



def generate_turns(entity):
    all_action_series = []
    theoretical_turn_length = len(free_subactions) + 6 + 1 + 1

    for length in range(1, 9):  # Turns can be 1 to 8 actions long: 6 Moves, 1 Object Interaction, 1 Action
                                        # new problem: they can contain any number of unique free actions...
        for turn in itertools.product(range(len(subaction_dict.keys())), repeat=length):
            if validate_turn(turn,entity):
                yield turn
    
    return list(all_action_series)


def validate_turn(turn,entity):
    # options 0, 1, and 2 are now Move Variants
    # a player can only move 6 times in a turn
    # 0 uses 1 move, 1 uses 2 moves, 2 uses 3 moves
    # options 3 and 8 are now Object Interactions
    
    move_count = 0
    individual_move_count = 0
    object_interact_count = 0
    action_count = 0
    bonus_action_count = 0
    attack_count = 0


    for action in turn:
        if action == 0:
            move_count += 1
            individual_move_count += 1

        if action == 1:
            move_count += 2
        
        if action == 2:
            move_count += 3
        
        if action == 3:
            move_count += 4

        if action in [4, 7]:
            object_interact_count += 1
        
        if action in action_subactions:
            action_count += 1
        
        if action in bonus_subactions:
            bonus_action_count += 1

        if action in attack_subactions:
            attack_count += 1

    return (move_count <= 6 and 
            individual_move_count <= 3 and
            object_interact_count <= (2 if 7 in turn else 1) and 
            action_count <= 1 and 
            bonus_action_count <= 1 and 
            attack_count <= entity.attack_limit)

def remove_redundant_turns(entity):
    turns = generate_turns(entity)
    solid_turns = list(turns)

    # what types of moves are redundant?
    # any combo that contains either 3 or 6 but not both and is otherwise the same is redundant
    # first, lets create a list of all the unique permutations
    #turns = turns
    # then, lets create a list of all the permutations that contain either 3 or 6
    #object_interact_turns = [turn for turn in turns if 3 in turn and 6 not in turn or 6 in turn and 3 not in turn]
    # any permutations that are the same other than 3 and 6 should be isolated
    #object_interact_turns = [turn for turn in object_interact_turns if turn not in [x for x in turns if x not in object_interact_turns]]
    
    # then I only need to keep one of the two
    #for turn in object_interact_turns:
    #    if 3 in turn:
    #        turns.remove([x for x in turns if 6 in x and x != turn][0])
    #    elif 6 in turn:
    #        turns.remove([x for x in turns if 3 in x and x != turn][0])

    # any turns that contain three adjacent 0s, two adjacent 0s, or a 0 and a 1 are redundant
    turns_to_remove = []

    for turn in solid_turns:
        adjacent_0_count = 0
        adjacent_1_count = 0
        adjacent_0_1_count = 0
        adjacent_0_2_count = 0
        adjacent_1_2_count = 0

        for i in range(len(turn)-1):
            
            if turn[i] == 0 and turn[i+1] == 0:
                adjacent_0_count += 1
            
            if turn[i] == 1 and turn[i+1] == 1:
                adjacent_1_count += 1
            
            if turn[i] == 0 and turn[i+1] == 1 or turn[i] == 1 and turn[i+1] == 0:
                adjacent_0_1_count += 1
            
            if turn[i] == 0 and turn[i+1] == 2 or turn[i] == 2 and turn[i+1] == 0:
                adjacent_0_2_count += 1

            if turn[i] == 1 and turn[i+1] == 2 or turn[i] == 2 and turn[i+1] == 1:
                adjacent_1_2_count += 1

        if adjacent_0_count > 0:
            turns_to_remove.append(turn)
        if adjacent_1_count > 0:
            turns_to_remove.append(turn)
        if adjacent_0_1_count > 0:
            turns_to_remove.append(turn)
        if adjacent_0_2_count > 0:
            turns_to_remove.append(turn)
        if adjacent_1_2_count > 0:
            turns_to_remove.append(turn)

    # remove duplicates in turns_to_remove
    turns_to_remove = list(set(turns_to_remove))

    for turn in turns_to_remove:
        solid_turns.remove(turn)

    return solid_turns



def choose_turn(entity):
    non_reduntant_turns = remove_redundant_turns(entity)
    all_action_series = {i:turn for i,turn in enumerate(non_reduntant_turns) if validate_turn(turn,entity)}
    print(len(all_action_series))
    return all_action_series

In [ ]:
def apply_permutations_to_location_series3(entity, action_series_dict, location_series_dict, one_space_permutations, two_space_permutations, three_space_permutations, four_space_permutations):
    # defining each element in the function:
    #   action_series                 - a set of numbers which each represent a sub-action an entity can take
    #   action_series_dict            - a dictionary containing all unique and legal action_series
    #   location_series               - a set of multiple (x,y) coordinates which represent the locations being targeted by the associated action_series
    #   permtuation_set               - the full power set of permutations of location_series, including illegal and illogical ones
    #   location_series_dict          - a dictionary containing all unique and legal location_series for the matching action_series from action_series_dict


    # what needs to happen in this function is for each action_series stored in action_series_dict,
    # each location_series in the appropriate permutation_set should be evaluated if it's appropriate for that action_series,
    # and if it meets all the criteria it should be stored in the location_series_dict
    # lastly, the function returns the finished location_series_dict


    # the current criteria that each location_series needs to meet:
    #   - the location_series must have the same number of locations as the associated action_series requires (i.e. length must match length)
    #   - the first location cannot be further away from the entity's starting position than the first move_action (either 1-4 spaces)
    #   - locations associated with the attack sub_action (currently represented by 5) cannot be more than 1 space away
    #   - locations associated with the attack sub_action cannot not be (or must be) in the enemy_locations list (because you can't target an empty space...yet)

    # the eventual criteria that each location_series should meet:
    #   - cannot move through obstructed spaces (requires cell states to be implemented)




    def determine_permutation_series(key, action_series, action_series_dict, one_space_permutations, two_space_permutations, three_space_permutations, four_space_permutations):
        # if the action_series contains 0 and not 1, 2, or 3, 
        # --and thus the entity is only moving a maximum of 1 space-- 
        # use one_space_permutations (the list of permutations for spaces within 1 space)
        
        if 0 in action_series_dict[key] and 1 not in action_series_dict[key] and 2 not in action_series_dict[key] and 3 not in action_series_dict[key]:
            permutation_set = one_space_permutations

        # if the action_series contains 1 and not 2 or 3, 
        # --and thus the entity is moving a maximum of 2 spaces--
        # then use two_space_permutations (the list of permutations for spaces within both 1 and 2 spaces)

        elif 1 in action_series_dict[key] and 2 not in action_series_dict[key] and 3 not in action_series_dict[key]:
            permutation_set = two_space_permutations

        # if the action_series contains 2 and not 3, 
        # --and thus the entity is moving a maximum of 3 spaces--
        # then use three_space_permutations (the list of permutations for spaces within 1, 2, and 3 spaces)

        elif 2 in action_series_dict[key] and 3 not in action_series_dict[key]:
            permutation_set = three_space_permutations

        # otherwise, use four_space_permutations
        else:
            permutation_set = four_space_permutations
        
        return permutation_set

    enemy_locations = entity.world.enemy_locations
    entity_location = entity.location
    coin_locations = entity.world.coin_locations

    for key, action_series in action_series_dict.items():

        permutation_set = determine_permutation_series(key, action_series, action_series_dict, one_space_permutations, two_space_permutations, three_space_permutations, four_space_permutations)
        number_of_locations_needed = sum(action_series.count(i) for i in subactions_req_targets)
        targeting_actions_only = list([x for x in action_series if x in subactions_req_targets])


        # first condition: length of location series must match length of number of locations needed
        filtered_permutation_set = [location_series for location_series in permutation_set if len(location_series) == number_of_locations_needed]
        

        for location_series in filtered_permutation_set:
            is_valid = True

            act_loc_zip = list(zip(targeting_actions_only,location_series))

            if act_loc_zip != []: # aka if there are locations being targeted

                # second condition: the first location cannot be distanced further away than allowable 
                if act_loc_zip[0][0] != 5 and chebyshev_distance(entity_location, act_loc_zip[0][1]) > act_loc_zip[0][0] + 1:
                    is_valid = False
                    continue

                # second condition: the first location cannot be distanced further away than allowable 
                elif act_loc_zip[0][0] == 5 and chebyshev_distance(entity_location, act_loc_zip[0][1]) > 1:
                    is_valid = False
                    continue

                # third condition: if the attack action is included, the associated location must be in enemy locations
                if act_loc_zip[0][0] == 5 and act_loc_zip[0][1] not in enemy_locations:
                    is_valid = False
                    continue


                # second condition: 
                if act_loc_zip[0][0] != 7 and chebyshev_distance(entity_location, act_loc_zip[0][1]) > act_loc_zip[0][0] + 1:
                    is_valid = False
                    continue

                # second condition: the first location cannot be distanced further away than allowable 
                elif act_loc_zip[0][0] == 7 and chebyshev_distance(entity_location, act_loc_zip[0][1]) > 1:
                    is_valid = False
                    continue

                # fourth condition: if object interaction (action) (7) is included, the associated location must be in coin_locations
                if act_loc_zip[0][0] == 7 and act_loc_zip[0][1] not in coin_locations or act_loc_zip[0][0] == 7 and act_loc_zip[0][1] != entity_location:
                    # using the object interaction action on the space the entity is in will be the shortcut for changing weapons...
                    is_valid = False
                    continue


                # second condition: 
                if act_loc_zip[0][0] != 4 and chebyshev_distance(entity_location, act_loc_zip[0][1]) > act_loc_zip[0][0] + 1:
                    is_valid = False
                    continue

                # second condition: the first location cannot be distanced further away than allowable 
                elif act_loc_zip[0][0] == 4 and chebyshev_distance(entity_location, act_loc_zip[0][1]) > 1:
                    is_valid = False
                    continue

                # fourth condition: if object interaction (action) (7) is included, the associated location must be in coin_locations
                if act_loc_zip[0][0] == 4 and act_loc_zip[0][1] not in coin_locations or act_loc_zip[0][0] == 4 and act_loc_zip[0][1] != entity_location:
                    # using the object interaction action on the space the entity is in will be the shortcut for changing weapons...
                    is_valid = False
                    continue

                
                # if first action is hide, if the space is bright or dim light, not allowed
                if act_loc_zip[0][0] == 9:
                    if entity.world.grid2[entity_location[0]][entity_location[1]][3] in [0,1]:
                        is_valid = False
                        continue


                # can't take the cast action if not a caster
                if 11 in action_series and entity.caster == False:
                    is_valid = False
                    continue


                # can't don or doff a shield if they don't have one in their inventory
                if 13 in action_series and entity.inventory == []:
                    is_valid = False
                    continue


                # can't end concentration if you're not concentrating in the first place
                if 14 in action_series and entity.concentrating == False:
                    is_valid = False
                    continue

                # can't take a bonus action attack if you don't have two weapons in hand
                    # will need to change this so that it can be equipped mid-turn and still valid...
                if 15 in action_series and len(entity.weapon_equipped) != 2:
                    is_valid = False
                    continue

                # third condition: if the attack action is included, the associated location must be in enemy locations
                if act_loc_zip[0][0] == 15 and act_loc_zip[0][1] not in enemy_locations:
                    is_valid = False
                    continue

                # need an ally to help in order to take the help action
                if 10 in action_series and len(entity.world.non_enemies) == 0:
                    is_valid = False
                    continue


                if len(act_loc_zip) > 1:

                    if 5 in action_series:
                        # second condition: the first location cannot be distanced further away than allowable 
                        five_index = targeting_actions_only.index(5)
                        if five_index == 0:
                            if chebyshev_distance(entity_location, act_loc_zip[1][1]) > act_loc_zip[1][0] + 1:
                                is_valid = False
                                continue

                        if five_index > 0:
                            if chebyshev_distance(act_loc_zip[five_index - 1][1],act_loc_zip[five_index][1]) > 1:
                                is_valid = False
                                continue

                            if act_loc_zip[five_index][1] not in enemy_locations:
                                is_valid = False
                                continue

                    if 7 in action_series:
                        # fourth condition: if object interaction (action) (7) is included, the associated location must be in coin_locations
                        seven_index = targeting_actions_only.index(7)
                        if seven_index == 0:
                            if chebyshev_distance(entity_location, act_loc_zip[1][1]) > act_loc_zip[1][0] + 1:
                                is_valid = False
                                continue
                            
                            if seven_index > 0:
                                if chebyshev_distance(act_loc_zip[seven_index - 1][1],act_loc_zip[seven_index][1]) > 1:
                                    is_valid = False
                                    continue

                                if act_loc_zip[seven_index][1] not in coin_locations and act_loc_zip[0][1] != entity_location:
                                    is_valid = False
                                    continue

                    if 4 in action_series:
                        # fourth condition: if object interaction (action) (7) is included, the associated location must be in coin_locations
                        four_index = targeting_actions_only.index(4)
                        if four_index == 0:
                            if chebyshev_distance(entity_location, act_loc_zip[1][1]) > act_loc_zip[1][0] + 1:
                                is_valid = False
                                continue
                            
                            if four_index > 0:
                                if chebyshev_distance(act_loc_zip[four_index - 1][1],act_loc_zip[four_index][1]) > 1:
                                    is_valid = False
                                    continue

                                if act_loc_zip[four_index][1] not in coin_locations and act_loc_zip[0][1] != entity_location:
                                    is_valid = False
                                    continue


                    if 9 in action_series:
                        targeting_actions_only_plus_nine = list([x for x in action_series if x in subactions_req_targets or x == 9])
                        nine_index = targeting_actions_only_plus_nine.index(9)
                        # I need the last location prior to the 9 
                        # if the last location prior to the 9 is in Bright or Dim light, false
                        if nine_index > 0:
                            new_location_act = targeting_actions_only_plus_nine[nine_index-1]
                            new_location_act_index = targeting_actions_only.index(new_location_act)
                            new_location_loc = location_series[new_location_act_index]

                            if entity.world.grid2[new_location_loc[0]][new_location_loc[1]][3] in [0,1]:
                                is_valid = False
                                continue

                    if 10 in action_series:
                        # help action
                        pass




            if is_valid:
                location_series_dict[key].append(location_series)

    return location_series_dict



In [ ]:
def calc_reward_for_action_series_location_series_combo2(entity,eval_action_series,eval_location_series):
    reward = 0

    # I want a random 50 50 chance that +1 is added to reward to test if different values can be assigned to the reward lists
    #probability = 0.5
    #reward += (1 if random.random() < probability else 0)

    targeting_actions_only = list([x for x in eval_action_series if x in subactions_req_targets])
    act_loc_zip = list(zip(targeting_actions_only,eval_location_series))
    enemy_locations = entity.world.enemy_locations
    #coin_locations = entity.world.coin_locations
    
    enemy_adjacent_locations = []
    for enemy_location in enemy_locations:
        enemy_adjacent_locations.append(adjacent_locations(enemy_location))

    
    # - if the entity attacks a creature, rewarded
    if 5 in eval_action_series:
        reward += 1


    # if an attack would reduce the target to 0 hp, reward
    damage = 1
    #if 5 in eval_action_series:
    #    attack_index = targeting_actions_only.index(5)
    #    enemy_index = enemy_locations.index(act_loc_zip[attack_index][1])
    #    enemy_name = f'enemy_{enemy_index}'
    #    if enemy_name.hp - damage <= 0:
    #        reward += 2


    # - if movement prompts an opportunity attack, punished

    move_series_zip = [(act,loc) for act, loc in act_loc_zip if act in move_subactions]
    move_path = calculate_full_path(entity.location, move_series_zip)

    if 8 not in eval_action_series: # 8 being the disengage action
        if check_opportunity_attacks(move_path, enemy_locations, disengage_action=8):
            reward -= 1

    # - if the same locations are being passed over, punished slightly

    if len(move_path) != len(set(move_path)):
        reward -= 1

    
    # - if movement is unoptimal, punished slightly (triangle method using length of sides)
        # the idea is that because there are only a maximum of 3 move subactions taken, a triangle can be drawn
        # however, in order for something to be "optimal" it has to have a goal
        # that goal can be determined by the actions in the action_series such as attack or object interaction
            # so if there is a 5, a more optimal move_series gets closer to an enemy 
            # if there's a 4 or 7, a more optimal move_series gets closer to objects
    
    visibility = check_visibility(entity.location, enemy_locations, entity.world)
    if visibility:
        reward -= 1
    else:
        reward += 1


    # - if an action isn't taken, punished
    action_bool = [x for x in eval_action_series if x in action_subactions]
    if len(action_bool) < 1:
        reward -= 1
    else:
        reward += 1

    # - punish for the number of enemies that have clear line of sight of the entity

    # - punish for the number of opportunity attacks that are prompted
        # punish extra if flanked for any of those opportunity attacks


    # - punish if entity ends turn flanked by multiple enemies 
    

    return reward
    

In [ ]:
def calc_risk_for_act_loc_zip():
    pass

    # what does this function need to accomplish?
    

    # what are extra rewards/penalties that can't currently be calculated
    # - killing an enemy
    # - dealing more damage
    # - avoiding taking damage
    # - chance the enemy hits you

    # - expected damage 


In [ ]:
def calc_rewards_for_location_series2(entity, reward_series_dict, location_series_dict, action_series_dict):

    for key in action_series_dict.keys():
        reward_series_dict[key] = [calc_reward_for_action_series_location_series_combo2(entity,action_series_dict[key],location_series) for location_series in location_series_dict[key]]

    return reward_series_dict

In [ ]:
def create_object_dict(action_series, location_series):
    
    targeting_actions_only = list([x for x in action_series if x in subactions_req_targets])
    act_loc_zip = list(zip(targeting_actions_only,location_series))

    # the object permutations will need to include every object in the inventory, equipped enemy weapons, and coins

    # three ways to make this work:
    # - based on action_series_dict values
        # so for every action_series in the dict, if object interaction (4 or 7) are included in action_series, then generate a set of locations equal to the number of object interactions that are taken
    # - based on location_series_dict values
        # if an object is in the location
    # - based on location_series per action_series


    # what would the ideal output look like
        # well for an action_series [0,4,1,7,0],
        # and the multiple location_series could be 
        #   [
        #           (0,1),(0,3),(0,4)
        #   ] 
        # 

    # the object permutations will need to be filtered by removing objects out of reach
    all_objects = []
    objects_within_range = []

    # object permutations would be a length of 2 at most

    # which should come first AKA how should possible object choices be represented: numeric index in all_objects or location?
    
    # or how could zip() be used here?
        # I could create a zip_act_loc_obj
    object_only_action_series = list([x for x in action_series if x in [4,7]])
    move_only_series = [x for x in action_series if x in move_subactions]
    for i in object_only_action_series:
        # for 4 or 7 in the action series
        subaction_index = action_series.index(i)
        targeting_action_before_subaction_index = action_series[subaction_index - 1]
        


    

In [ ]:
def create_dash_dict(entity, action_series_dict, action_series, location_series):
    dash_permutations = []
    dash_dict = {x for x in action_series_dict}

    starting_location = 'placeholder' # this is where the entity is starting from 

In [ ]:
def create_cast_dict():
    pass

In [ ]:

def generate_location_series(entity):
    # I could do a move_one_locations, move_two_locations, move_three_locations, etc.
    # each would be a list of spaces within the Chebyshev distance of 1, 2, 3, etc.

    timing_results = {}
    imported_previous_dict_gen = False

    try:
        start_time = time.time()

        previous_dict_gen = import_from_json('previous_dict_gen.json')
        print(previous_dict_gen)

        end_time = time.time()
        timing_results['Importing previous_dict_gen'] = end_time - start_time

        imported_previous_dict_gen = True

    except:
        
        print('No previous_dict_gen.json available')

    if imported_previous_dict_gen == True:
        same_parameters_as_jsons = False

        start_time = time.time()
        print('subaction length: ', len(entity.subaction_options_dict))
        print('entity location: ', entity.location)
        
        Bypass = True
        if previous_dict_gen['entity_location'] == entity.location and previous_dict_gen['enemy_locations'] == entity.world.enemy_locations and previous_dict_gen['subaction_num'] == len(entity.subaction_options_dict) or Bypass == True:

            action_series_dict = import_from_json('action_series_dict.json')
            location_series_dict = import_from_json('location_series_dict.json')
            reward_series_dict = import_from_json('reward_series_dict.json')
            subaction_dict = import_from_json('subaction_dict.json')

            same_parameters_as_jsons = True
        else:
            if previous_dict_gen['entity_location'] == entity.location:
                print('location change')
            
            if previous_dict_gen['enemy_locations'] == entity.world.enemy_locations:
                print('enemy location change')
            
            if previous_dict_gen['subaction_num'] == len(entity.subaction_options_dict):
                print('new action options')

            print('Different parameters as last time')
            
        end_time = time.time()
        timing_results['Importing previous series_dicts'] = end_time - start_time

    if same_parameters_as_jsons == False:





        start_time = time.time()
        locations_within_range = []
        for x in range(-6,7):
            for y in range(-6,7):
                if chebyshev_distance(entity.location,(x,y)) <= 6:
                    locations_within_range.append((x,y))

        move_one_locations = []
        move_two_locations = []
        move_three_locations = []
        move_four_locations = []
        move_five_locations = []
        move_six_locations = []

        for location in locations_within_range:
            if chebyshev_distance(entity.location,location) <= 1:
                move_one_locations.append(location)
            if chebyshev_distance(entity.location,location) == 2:
                move_two_locations.append(location)
            if chebyshev_distance(entity.location,location) == 3:
                move_three_locations.append(location)
            if chebyshev_distance(entity.location,location) == 4:
                move_four_locations.append(location)
            if chebyshev_distance(entity.location,location) == 5:
                move_five_locations.append(location)
            if chebyshev_distance(entity.location,location) == 6:
                move_six_locations.append(location)



        end_time = time.time()
        timing_results['Generate and categorize locations'] = end_time - start_time
        start_time = time.time()
        ##### Now for Creating the Location Series #####
        # the most location targets I'll need in a single turn is now 4: moves 1-4 and attack
        # so I only need to create variations that are up to 4 long

        # what if instead of creating all the permutations, I did several based on distance

        # permutations within 1 space
        #one_space_permutations = list(itertools.permutations(move_one_locations,4))
        one_space_permutations = list(itertools.chain.from_iterable(combinations(move_one_locations, r) for r in range(0, 4)))
        print('One Space Locations: ',len(move_one_locations))
        print('One Space Permutations: ',len(one_space_permutations))
        print('One Space Permutations: ',one_space_permutations)

        #print(one_space_permutations)

        # permutations within 2 spaces
        two_space_permutations = list(itertools.permutations(move_one_locations + move_two_locations,4))
        two_space_permutations = list(itertools.chain.from_iterable(combinations(move_one_locations + move_two_locations, r) for r in range(0, 4)))
        print('Two Space Locations: ',len(move_one_locations + move_two_locations))
        print('Two Space Permutations: ',len(two_space_permutations))
        print(two_space_permutations)

        # permutations within 3 spaces
        # the distance between the two locations 
        three_space_permutations = list(itertools.permutations(move_one_locations + move_two_locations + move_three_locations,4))
        three_space_permutations = list(itertools.chain.from_iterable(combinations(move_one_locations + move_two_locations + move_three_locations, r) for r in range(0, 4)))
        print('Three Space Locations: ',len(move_one_locations + move_two_locations + move_three_locations))
        print('Three Space Permutations: ',len(three_space_permutations))

        # permutations within 4 spaces
        four_space_permutations = list(itertools.permutations(move_one_locations + move_two_locations + move_three_locations + move_four_locations,4))
        four_space_permutations = list(itertools.chain.from_iterable(combinations(move_one_locations + move_two_locations + move_three_locations + move_four_locations, r) for r in range(0, 4)))
        print('Four Space Locations: ',len(move_one_locations + move_two_locations + move_three_locations + move_four_locations))
        print('Four Space Permutations: ',len(four_space_permutations))

        #permutations = []
        #for i in range(4):
        #    locations = sum(move_locations[:i+1], [])
        #    perms = list(itertools.chain.from_iterable(itertools.combinations(locations, r) for r in range(1, 5)))
        #    permutations.append(perms)


        end_time = time.time()
        timing_results['Generate permutations'] = end_time - start_time
        start_time = time.time()
    

        one_space_permutations = [x for x in one_space_permutations if (len(x) < 2 or chebyshev_distance(x[0], x[1]) <= 1) and (len(x) < 3 or chebyshev_distance(x[1], x[2]) <= 1) and (len(x) < 4 or chebyshev_distance(x[2], x[3]) <= 1)]
        two_space_permutations = [x for x in two_space_permutations if (len(x) < 2 or chebyshev_distance(x[0], x[1]) <= 2) and (len(x) < 3 or chebyshev_distance(x[1], x[2]) <= 2) and (len(x) < 4 or chebyshev_distance(x[2], x[3]) <= 2)]
        three_space_permutations = [x for x in three_space_permutations if (len(x) < 2 or chebyshev_distance(x[0], x[1]) <= 3) and (len(x) < 3 or chebyshev_distance(x[1], x[2]) <= 3) and (len(x) < 4 or chebyshev_distance(x[2], x[3]) <= 3)]
        four_space_permutations = [x for x in four_space_permutations if (len(x) < 2 or chebyshev_distance(x[0], x[1]) <= 4) and (len(x) < 3 or chebyshev_distance(x[1], x[2]) <= 4) and (len(x) < 4 or chebyshev_distance(x[2], x[3]) <= 4)]

        # - remove permutations that have more than:
        #  3 two-space jumps
        #  2 three-space jumps
        #  1 four-space jump  
        two_space_permutations = [x for x in two_space_permutations if sum([1 for i in range(len(x)-1) if chebyshev_distance(x[i],x[i+1]) == 2]) <= 3]
        three_space_permutations = [x for x in three_space_permutations if sum([1 for i in range(len(x)-1) if chebyshev_distance(x[i],x[i+1]) == 3]) <= 2]
        three_space_permutations = [x for x in three_space_permutations if sum([1 for i in range(len(x)-1) if chebyshev_distance(x[i],x[i+1]) == 2]) <= 3]
        four_space_permutations = [x for x in four_space_permutations if sum([1 for i in range(len(x)-1) if chebyshev_distance(x[i],x[i+1]) == 4]) <= 1]
        four_space_permutations = [x for x in four_space_permutations if sum([1 for i in range(len(x)-1) if chebyshev_distance(x[i],x[i+1]) == 3]) <= 2]
        four_space_permutations = [x for x in four_space_permutations if sum([1 for i in range(len(x)-1) if chebyshev_distance(x[i],x[i+1]) == 2]) <= 3]

        # - remove those with the same location repeated
        one_space_permutations = [x for x in one_space_permutations if (len(x) < 2 or x[0] != x[1]) and (len(x) < 3 or x[1] != x[2]) and (len(x) < 4 or x[2] != x[3])]
        two_space_permutations = [x for x in two_space_permutations if (len(x) < 2 or x[0] != x[1]) and (len(x) < 3 or x[1] != x[2]) and (len(x) < 4 or x[2] != x[3])]
        three_space_permutations = [x for x in three_space_permutations if (len(x) < 2 or x[0] != x[1]) and (len(x) < 3 or x[1] != x[2]) and (len(x) < 4 or x[2] != x[3])]
        four_space_permutations = [x for x in four_space_permutations if (len(x) < 2 or x[0] != x[1]) and (len(x) < 3 or x[1] != x[2]) and (len(x) < 4 or x[2] != x[3])]



        end_time = time.time()
        timing_results['Filter permutations'] = end_time - start_time
        start_time = time.time()

        print('')
        print('Filtered One Space Permutations: ',len(one_space_permutations))
        print('Filtered Two Space Permutations: ',len(two_space_permutations))
        print('Filtered Three Space Permutations: ',len(three_space_permutations))
        print('Filtered Four Space Permutations: ',len(four_space_permutations))

        print('')
        print(one_space_permutations[:10])
        print(two_space_permutations[10:20])
        print(three_space_permutations[20:30])
        print(four_space_permutations[100:120])



        action_series_dict = choose_turn(entity)
        end_time = time.time()
        timing_results['Create action_series_dict'] = end_time - start_time
        start_time = time.time()

        
        # now to create the location_series_dict
        location_series_dict = {x for x in action_series_dict.keys()}
        location_series_dict = {x:[] for x in location_series_dict}


        # apply_permutations_to_location_series(action_series_dict, one_space_permutations)
        location_series_dict = apply_permutations_to_location_series3(entity, action_series_dict, location_series_dict, one_space_permutations, two_space_permutations, three_space_permutations, four_space_permutations)

        end_time = time.time()
        timing_results['Create location_series_dict'] = end_time - start_time
        start_time = time.time()


        # Object Dictionary
        # Cast Dictionary
        # Dash Dictionary
        # 


        # now I need to make the reward dictionary
        reward_series_dict = {x for x in action_series_dict.keys()}
        reward_series_dict = {x:[] for x in location_series_dict}

        reward_series_dict = calc_rewards_for_location_series2(entity, reward_series_dict, location_series_dict, action_series_dict)
        
        end_time = time.time()
        timing_results['Create reward_series_dict'] = end_time - start_time
        start_time = time.time()

        subaction_dict = entity.subaction_options_dict

        stored_data = { # a dictionary to check before loading the rest of the jsons
            'entity_location': entity.location,
            'enemy_locations': entity.world.enemy_locations,
            'subaction_num': len(subaction_dict),
        }
        
        export_to_json(stored_data, 'previous_dict_gen.json')
        export_to_json(action_series_dict, 'action_series_dict.json')
        export_to_json(location_series_dict, 'location_series_dict.json')
        export_to_json(reward_series_dict, 'reward_series_dict.json')

        export_to_json(subaction_dict, 'subaction_dict.json')

        end_time = time.time()
        timing_results['Export Dictionaries'] = end_time - start_time

    print("\nTiming results:")
    for step, duration in timing_results.items():
        print(f"{step}: {duration:.4f} seconds")
    


    return action_series_dict, location_series_dict, reward_series_dict

#action_series_dict, location_series_dict, reward_series_dict = generate_location_series(actor)





In [ ]:
#action_series_dict

In [ ]:
#print('Location Series: ',sum(len(value) for value in location_series_dict.values()))
#print(location_series_dict)


#for i in range(len(location_series_dict)):
#    print(f'{i}: {location_series_dict[i]}')

In [ ]:
#print('Reward Series: ',sum(len(value) for value in reward_series_dict.values()))
#print('Highest Value: ', {max(max(value, default=-float('inf')) for value in reward_series_dict.values())})
#print('Lowest Value: ', {min(min(value, default=float('inf')) for value in reward_series_dict.values())})

#print(' ')

#print('Empty Reward Series: ',sum())
#for i in range(len(reward_series_dict)):
#    print(f'{i}: {reward_series_dict[i]}')

In [ ]:
#def process_act_loc_zip(entity, act_loc_zip):
#    pass

In [ ]:
#Choose_Turn(stage, actor, False)

# takes 20 minutes to process 261/650